# 230623_sub_causal_tracing

submodule 단위로 가능하도록 변경

# Import libraries

In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer, set_seed
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
import datasets
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from datetime import date
import sys
import os


/rds/general/user/jj1122/home/anaconda3/envs/m2d2_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
torch.manual_seed(718)
set_seed(718)

In [3]:
job_cd = "omcd_econ_l1"
# job_cd = "tmod_Religion_and_belief_systems"
# job_cd = sys.argv[2] # TODO
list_job_cd = job_cd.split("_")
job_gubun = list_job_cd[0]
model_id = "_".join(list_job_cd[1:])

In [4]:
# job_cd = sys.argv[2]
job_cd = "omcd_econ_l1"

list_job_cd = job_cd.split("_")
job_gubun = list_job_cd[0]
model_id = "_".join(list_job_cd[1:])
print(job_cd, job_gubun, model_id)

assert job_gubun in ["omcd", "tmod"]

cache_dir = "/rds/general/user/jj1122/ephemeral/.cache/huggingface"
device_id = "cuda"

# n_layers = 12
# list_modules = ['attn', 'mlp']
# trace_module_id = "transformer.h.{l}.{m}"

tuned_model_path = f"/rds/general/user/jj1122/home/projects/m2d2/dataset/{model_id}/models"

today_dt = date.today().strftime("%y%m%d")
output_file = f"/rds/general/user/jj1122/home/projects/m2d2/utils/output_logs/{today_dt}_sub_{job_gubun}_{model_id}.json"


omcd_econ_l1 omcd econ_l1


# DATA

## Dataset

In [5]:
# df_dataset = pd.read_csv("./data/dataset_info_230709.tsv", sep="\t")

In [6]:
# list_dataset_nms =df_dataset.loc[(df_dataset.category == "L1") & (df_dataset.editing == True), "dataset"].tolist()

## Submodule

In [7]:
cache_dir = "/rds/general/user/jj1122/ephemeral/.cache/huggingface"

def parse_code_submodule(row):
    list_code = row.code.split(".")
    row["trace_id"] = ".".join(row["code"].split(".")[:-1])
#     row["component_id"] = ".".join(row["code"].split(".")[1:-1])
    is_in_layer = row["code"].startswith("transformer.h")
    

    if is_in_layer:        
        row["layer"] = int(list_code[2])
        row["module"] = list_code[3]
    else:
        row["module"] = list_code[1]
        if row["module"] == "ln_f": 
            row["layer"] = int(99)
        elif row["module"] in ["wte", "wpe"]:
            row['layer'] = int(-1)
    
    if row["module"] in ["attn", "mlp"]:
        row["submodule"] = list_code[-2]
    else:
        row["submodule"] = row["module"]

    row["w_or_b"] = list_code[-1]
    row["is_in_layer"] = is_in_layer
    
    is_investigated = (not row["code"].startswith("transformer.w")) and (not row["submodule"].startswith("ln"))
    row["is_investigated"] = is_investigated

    return row


dict_n_parmas = {tup[0]: tup[1].numel() for tup in GPT2LMHeadModel.from_pretrained("gpt2", cache_dir=cache_dir).to("cpu").named_parameters()}
df_submodule = pd.DataFrame.from_dict(dict_n_parmas, orient="index").reset_index()
df_submodule.columns = ["code", 'params']
df_submodule = df_submodule.apply(lambda row: parse_code_submodule(row), axis=1)
df_temp = df_submodule.groupby("trace_id")["params"].sum().to_frame().reset_index()
df_temp.columns = ["trace_id", "total"]

df_submodule["display_id"] = df_submodule["layer"].astype(str) + "." + df_submodule["module"]

df_submodule.loc[df_submodule.module != df_submodule.submodule, "display_id"] \
    = df_submodule.loc[df_submodule.module != df_submodule.submodule, "display_id"] \
        + "." + df_submodule.loc[df_submodule.module != df_submodule.submodule, "submodule"]

df_submodule = df_submodule.merge(df_temp, on=["trace_id"])
list_trace_ids = df_submodule.loc[df_submodule.is_investigated].trace_id.unique()

# Data

In [8]:
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=cache_dir)
dataset = load_dataset("machelreid/m2d2", model_id, cache_dir=cache_dir)
ds_test = dataset["test"].filter(lambda x: x['text'] != '')


Found cached dataset m2d2 (/rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/econ_l1/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00)
100%|██████████| 3/3 [00:00<00:00,  7.58it/s]
Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/econ_l1/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-1eec3cf03a698048.arrow


In [9]:
set_seed(718)

def tokenize_function(examples):
    output = gpt2_tokenizer(examples['text'], max_length=1024, truncation=True)
    return output

tokenized_datasets = ds_test.map(
    tokenize_function,
    batched=True,
    num_proc=8,
    remove_columns='text', #TODO
    load_from_cache_file=True,
)

len_sentences = len(tokenized_datasets)

Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/econ_l1/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-12826ea30f123b74_*_of_00008.arrow


# MODEL & HOOK

In [15]:
list_trace_ids

array(['transformer.h.0.attn.c_attn', 'transformer.h.0.attn.c_proj',
       'transformer.h.0.mlp.c_fc', 'transformer.h.0.mlp.c_proj',
       'transformer.h.1.attn.c_attn', 'transformer.h.1.attn.c_proj',
       'transformer.h.1.mlp.c_fc', 'transformer.h.1.mlp.c_proj',
       'transformer.h.2.attn.c_attn', 'transformer.h.2.attn.c_proj',
       'transformer.h.2.mlp.c_fc', 'transformer.h.2.mlp.c_proj',
       'transformer.h.3.attn.c_attn', 'transformer.h.3.attn.c_proj',
       'transformer.h.3.mlp.c_fc', 'transformer.h.3.mlp.c_proj',
       'transformer.h.4.attn.c_attn', 'transformer.h.4.attn.c_proj',
       'transformer.h.4.mlp.c_fc', 'transformer.h.4.mlp.c_proj',
       'transformer.h.5.attn.c_attn', 'transformer.h.5.attn.c_proj',
       'transformer.h.5.mlp.c_fc', 'transformer.h.5.mlp.c_proj',
       'transformer.h.6.attn.c_attn', 'transformer.h.6.attn.c_proj',
       'transformer.h.6.mlp.c_fc', 'transformer.h.6.mlp.c_proj',
       'transformer.h.7.attn.c_attn', 'transformer.h.7.attn.c_

In [10]:
# MODEL & HOOK
def load_model(model="tuned"):
    if model == "tuned":
        return GPT2LMHeadModel.from_pretrained(tuned_model_path).to(device_id)
    else:
        return GPT2LMHeadModel.from_pretrained(model, cache_dir=cache_dir).to(device_id)


def save_clean_activation(m_id):
    def save_clean_activation_hook(module, _input, _output):
        clean_activations[m_id] = _output.detach()
    return save_clean_activation_hook


def corrupt_input_vector_v2(module, _input, _output):
    torch.manual_seed(718)
    std = torch.std(_output)
    output = _output + (std*1.5) * torch.randn(_output[0].shape).to(device_id)
    return output


def restore_activation(m_id):
    def restore_activation_hook(module, _input, _output):
        clean_activation = clean_activations[m_id]#[:, t]
        new_output = clean_activation
        del clean_activation
        print(f"m_id: {m_id}")
        print(f"module: {module}")
        print(f"_input: {_input}")
        print(f"_output: {_output}")
        print(f"new_output: {new_output}")
        return new_output 
    return restore_activation_hook

## Register Hook

In [11]:
set_seed(718)

if job_gubun == "omcd":
    clean_model = load_model()
    clean_model.eval()
    for m_id in list_trace_ids:
#         print(m_id)
        clean_model.get_submodule(m_id).register_forward_hook(save_clean_activation(m_id))    

    # Second run: corrupted run def
    corrupted_model = load_model()
    corrupted_model.eval()
    corrupted_model.get_submodule("transformer.wte").register_forward_hook(corrupt_input_vector_v2)

    # Third run: restored run def    
    restored_model = load_model()
    restored_model.eval()
    restored_model.get_submodule("transformer.wte").register_forward_hook(corrupt_input_vector_v2)
else:
    clean_model = load_model()
    clean_model.eval()
    for m_id in list_trace_ids:
        clean_model.get_submodule(m_id).register_forward_hook(save_clean_activation(m_id))

    # Second run: base run def
    corrupted_model = load_model("gpt2")
    corrupted_model.eval()

    # Third run: restored run def
    restored_model = load_model("gpt2")
    restored_model.eval()


# Causal Tracing

In [12]:
try:
    with open(output_file, 'r') as json_file:
        list_results = json.load(json_file)
    for i, d in enumerate(list_results):
        if len(d) == 0:
            done_idx = i - 1
            break
except:
    list_results = [{} for x in range(len_sentences)]
    done_idx = 0
print(done_idx)

0


In [13]:
done_idx = 0

In [17]:
clean_activations["transformer.h.0.attn.c_attn"].shape

torch.Size([1, 91, 2304])

In [18]:
clean_activations["transformer.h.0.attn.c_proj"].shape

torch.Size([1, 91, 768])

In [19]:
clean_activations["transformer.h.0.mlp.c_fc"].shape

torch.Size([1, 91, 3072])

In [20]:
clean_activations["transformer.h.0.mlp.c_proj"].shape

torch.Size([1, 91, 768])

In [26]:
torch.manual_seed(718)
set_seed(718)

for sentence_idx, data in enumerate(tokenized_datasets):
    print(f"sentence_idx: {sentence_idx}")
    if sentence_idx == 1: break
    
    if done_idx > sentence_idx: continue

#     if sentence_idx % 1000 == 0:
#         print(f"sentence_idx: {sentence_idx}")
#         with open(output_file, 'w') as json_file:
#             json.dump(list_results, json_file)

    inputs = torch.tensor(data['input_ids']).to(device_id)

    # First run: clean run
    clean_activations = {}
    with torch.no_grad():
        clean_outputs = clean_model(inputs, labels=inputs.clone())
        clean_loss = np.exp(clean_outputs.loss.item())

    # Second run: corrupted run
    with torch.no_grad():
        corrupted_outputs = corrupted_model(inputs, labels=inputs.clone())
        corrupted_loss = np.exp(corrupted_outputs.loss.item())

#     Third run: corrupted-with-restoration run
    restored_loss = {}
    with torch.no_grad():
        for m_id in list_trace_ids:
#             print(f"m_id: {m_id}")
#             restored_model = load_model("gpt2")
#             restored_model.eval()
#             restored_model.get_submodule("transformer.wte").register_forward_hook(corrupt_input_vector_v2)
#             restored_model.get_submodule(m_id).register_forward_hook(restore_activation(m_id))

            hook = restored_model.get_submodule(m_id).register_forward_hook(restore_activation(m_id))

            restored_outputs = restored_model(inputs, labels=inputs.clone())
        
            restored_loss[m_id] = np.exp(restored_outputs.loss.item())
            print("restored_outputs.logits:", restored_outputs.logits)
            print("restored_loss:", restored_loss[m_id])
            
            hook.remove()
            restored_model.reset_memory_hooks_state()
#             print([x[0] for x in restored_model.named_buffers()])
            print()

    list_results[sentence_idx]['clean_loss'] = clean_loss
    list_results[sentence_idx]['corrupted_loss'] = corrupted_loss
    list_results[sentence_idx]['restored_loss'] = restored_loss

sentence_idx: 0
m_id: transformer.h.0.attn.c_attn
module: Conv1D()
_input: (tensor([[[-0.0615, -0.1206, -0.1294,  ..., -0.0864,  0.0021,  0.0195],
         [ 0.4367, -0.0484,  0.0263,  ...,  0.1548,  0.2173,  0.1258],
         [-0.4231, -0.1931, -0.0902,  ...,  0.0283,  0.1649, -0.0793],
         ...,
         [ 0.2313, -0.1444,  0.1054,  ..., -0.2182,  0.0458,  0.1173],
         [-0.3861, -0.1854,  0.0642,  ...,  0.2437,  0.0576, -0.1109],
         [-0.2373,  0.0031,  0.0299,  ..., -0.1507,  0.1637,  0.4065]]],
       device='cuda:0'),)
_output: tensor([[[-0.8298, -0.6448, -0.4022,  ..., -0.0069, -0.1674,  0.0688],
         [-0.7464,  0.9376, -1.3309,  ...,  0.1358,  0.4994,  0.2281],
         [ 0.6109,  0.1870, -0.8229,  ...,  0.2682,  0.0185,  0.1607],
         ...,
         [ 0.7141, -0.3613, -1.9945,  ...,  0.4382,  0.0813,  0.2043],
         [ 1.6572,  0.6914, -0.4856,  ...,  0.0381, -0.3873, -0.0249],
         [-0.1834, -0.2926,  0.0105,  ..., -0.0664,  0.1361, -0.3783]]],
     

In [86]:
list_results[0]["restored_loss"]

{'transformer.h.0.attn.c_attn': 29.470737374030648,
 'transformer.h.0.attn.c_proj': 29.470737374030648,
 'transformer.h.0.mlp.c_fc': 25.420941218783344,
 'transformer.h.0.mlp.c_proj': 25.420941218783344,
 'transformer.h.1.attn.c_attn': 88.38348771047524,
 'transformer.h.1.attn.c_proj': 88.38348771047524,
 'transformer.h.1.mlp.c_fc': 77.14880016903861,
 'transformer.h.1.mlp.c_proj': 77.14880016903861,
 'transformer.h.2.attn.c_attn': 69.19004109179598,
 'transformer.h.2.attn.c_proj': 69.19004109179598,
 'transformer.h.2.mlp.c_fc': 65.11407848586268,
 'transformer.h.2.mlp.c_proj': 65.11407848586268,
 'transformer.h.3.attn.c_attn': 68.12920548521868,
 'transformer.h.3.attn.c_proj': 68.12920548521868,
 'transformer.h.3.mlp.c_fc': 66.82736649478748,
 'transformer.h.3.mlp.c_proj': 66.82736649478748,
 'transformer.h.4.attn.c_attn': 64.28147114724902,
 'transformer.h.4.attn.c_proj': 64.28147114724902,
 'transformer.h.4.mlp.c_fc': 64.8539447192554,
 'transformer.h.4.mlp.c_proj': 64.8539447192554

In [63]:
clean_activations["transformer.h.0.attn.c_attn"]

tensor([[[-0.0288, -0.3114,  0.0557,  ...,  0.0645, -0.1732,  0.0111],
         [-0.4986,  0.1701, -1.0163,  ...,  0.3744,  0.3668,  0.1815],
         [ 0.3047,  0.8006,  0.1433,  ...,  0.1129,  0.2242,  0.0319],
         ...,
         [ 1.0819, -0.0811, -0.6493,  ...,  0.4032,  0.0125,  0.2486],
         [ 1.6623, -0.3283,  0.1893,  ..., -0.0898,  0.0466,  0.0422],
         [ 0.3081, -1.8928,  0.2130,  ..., -0.0200,  0.1305, -0.0184]]],
       device='cuda:0')

In [31]:
clean_activations["transformer.h.0.attn.c_proj"]

tensor([[[ 1.0769, -0.2054,  1.3931,  ..., -0.1546,  0.0732, -0.0331],
         [ 0.4085,  0.3696,  0.8274,  ..., -0.2024,  0.1198, -0.0257],
         [ 1.0333,  0.0755,  0.4720,  ..., -0.1324,  0.0962, -0.0324],
         ...,
         [ 0.3928, -0.6708, -0.5011,  ...,  0.0143,  0.0163,  0.0145],
         [-0.9803, -0.6987, -0.4718,  ..., -0.0120, -0.0051, -0.0035],
         [-0.1719,  0.3631, -0.5965,  ..., -0.0424, -0.0377,  0.0986]]],
       device='cuda:0')